In [28]:
import tensorflow as tf
#'tf.reset.default_graph()' to fix error use Xavier_initializer()
tf.reset_default_graph()
import random
from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [29]:
learning_rate = 0.001
epochs = 20
batch = 100

In [30]:
#Use Dropout = keep_prob, not max train(?)
keep_prob = tf.placeholder(tf.float32)

In [31]:
X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1]) #28*28*1, -1 means anything.
Y = tf.placeholder(tf.float32, [None, 10])

In [32]:
#filter 3 by 3, color = 1, number_of_filter = 32
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev = 0.01)) 

#padding = 'SAME' means, stride 1, Regardless of Weight_Size, output of L1 as same as input_image_size
#conv2d   -> (?, 28, 28, 32)
#max_pool -> (?, 14, 14, 32)
L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

#filter2 same as Input_size(32), filter2_size = 64
#Reshape : Unfold L2 to FC
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev = 0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding = 'SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L2 = tf.reshape(L2, [-1, 7*7*64])

#FC input(3136 = 7*7*64) -> 10 outputs
#Use Xavier_initializer
W3 = tf.get_variable("W3", shape=[7*7*64, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2,W3)+b

#Use Adamoptimizer (instead GradientDescentOptimizer)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [33]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('Start Learning')
for epoch in range(epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch)
        feed_dict = {X: batch_xs, Y: batch_ys}
        #save cost value in 'c'
        c, _, = sess.run([cost, optimizer], feed_dict = feed_dict)
        avg_cost += c / total_batch
    print('Epoch:' '%04d' %(epoch + 1), 'Cost = ', '{:.9f}'.format(avg_cost))
print('Finish Learning')

'''
Start Learning
Epoch:0001 Cost =  0.361479089
Epoch:0002 Cost =  0.093500431
Epoch:0003 Cost =  0.067787592
Epoch:0004 Cost =  0.056125973
Epoch:0005 Cost =  0.047706391
Epoch:0006 Cost =  0.041562431
Epoch:0007 Cost =  0.035755749
Epoch:0008 Cost =  0.030990488
Epoch:0009 Cost =  0.027794329
Epoch:0010 Cost =  0.025630914
Epoch:0011 Cost =  0.022249624
Epoch:0012 Cost =  0.019832688
Epoch:0013 Cost =  0.016631698
Epoch:0014 Cost =  0.015857259
Epoch:0015 Cost =  0.013306068
Epoch:0016 Cost =  0.012922972
Epoch:0017 Cost =  0.010538649
Epoch:0018 Cost =  0.008633421
Epoch:0019 Cost =  0.009345217
Epoch:0020 Cost =  0.008696372
Finish Learning
'''

Start Learning
Epoch:0001 Cost =  0.361479089
Epoch:0002 Cost =  0.093500431
Epoch:0003 Cost =  0.067787592
Epoch:0004 Cost =  0.056125973
Epoch:0005 Cost =  0.047706391
Epoch:0006 Cost =  0.041562431
Epoch:0007 Cost =  0.035755749
Epoch:0008 Cost =  0.030990488
Epoch:0009 Cost =  0.027794329
Epoch:0010 Cost =  0.025630914
Epoch:0011 Cost =  0.022249624
Epoch:0012 Cost =  0.019832688
Epoch:0013 Cost =  0.016631698
Epoch:0014 Cost =  0.015857259
Epoch:0015 Cost =  0.013306068
Epoch:0016 Cost =  0.012922972
Epoch:0017 Cost =  0.010538649
Epoch:0018 Cost =  0.008633421
Epoch:0019 Cost =  0.009345217
Epoch:0020 Cost =  0.008696372
Finish Learning


In [34]:
#big Y & hypothesis, and use tf.equal
#accuracy : Average of correct_prediction
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

#Accuracy: 0.9873, epoch=20, batch_size=100

Accuracy: 0.9873
